In [1]:
from flask import Flask, request, jsonify
import json
import torch
from transformers import BertTokenizer, BertModel
from ckiptagger import WS
from ultralytics import YOLO
from PIL import Image
import numpy as np
from collections import Counter
import torch.nn.functional as F

# 初始化 Flask
app = Flask(__name__)

# 初始化 CKIP、BERT、YOLO
ws = WS("C:\\Users\\user\\OneDrive\\桌面\\data")
tokenizer_zh = BertTokenizer.from_pretrained('bert-base-chinese')
bert_model_zh = BertModel.from_pretrained('bert-base-chinese')
yolo_model = YOLO("yolov8n.pt")

# BERT 嵌入計算
def get_bert_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].cpu().tolist()

# YOLO 偵測物件
@app.route('/detect_objects', methods=['POST'])
def detect_objects():
    data = request.json
    image_path = data.get('image_path')
    if not image_path:
        return jsonify({'error': '缺少圖片路徑'}), 400

    results = yolo_model(image_path)
    labels = [yolo_model.names[int(cls)] for cls in results[0].boxes.cls.tolist()]

    return jsonify({'labels': labels}), 200

# 文字 WS 與 BERT 嵌入
@app.route('/process_text', methods=['POST'])
def process_text():
    data = request.json
    text = data.get('text')
    if not text:
        return jsonify({'error': '缺少文本內容'}), 400

    tokens = ws([text])
    bert_embedding = get_bert_embedding(' '.join(tokens[0]), tokenizer_zh, bert_model_zh)

    return jsonify({'tokens': tokens[0], 'embedding': bert_embedding}), 200

if __name__ == "__main__":
    app.run(port=5001)


ModuleNotFoundError: No module named 'mysql'